In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

In [4]:
athletes = pd.read_csv('data/summerOly_athletes.csv')
hosts = pd.read_csv('data/summerOly_hosts.csv')
medal_counts = pd.read_csv('data/summerOly_medal_counts.csv')
programs = pd.read_csv('data/summerOly_programs.csv', encoding='utf-8')

In [5]:
medal_counts

,Rank,NOC,Gold,Silver,Bronze,Total,Year
0,1,United States,11,7,2,20,1896
1,2,Greece,10,18,19,47,1896
2,3,Germany,6,5,2,13,1896
3,4,France,5,4,2,11,1896
4,5,Great Britain,2,3,2,7,1896
...,...,...,...,...,...,...,...
1430,84,Qatar,0,0,1,1,2024
1431,84,Refugee Olympic Team,0,0,1,1,2024
1432,84,Singapore,0,0,1,1,2024
1433,84,Slovakia,0,0,1,1,2024


In [ ]:
continent_mapping = {
    "AF": [
        "ALG", "ANG", "BEN", "BIZ", "BOT", "BUR", "CAF", "CAM", "CHA", "CIV", "COM", "COD", "DJI", "EGY", "ERI", "ETH", "GAB", "GAM", "GHA", "GUI", "KEN", "LES", "LBR", "LBY", "MAD", "MAW", "MLI", "MLT", "MOZ", "MRI", "NAM", "NGR", "RWA", "SEN", "SEY", "SLE", "SOM", "STP", "SUD", "SWZ", "TAN", "TCH", "TOG", "TUN", "UGA", "ZAM", "ZIM"
    ],
    "AS": [
        "AFG", "AZE", "BHU", "BRN", "BRU", "BUR", "CAM", "CHN", "CYP", "GEO", "IND", "INA", "IOA", "IRQ", "IRL", "IRI", "ISR", "JOR", "JPN", "KAZ", "KGZ", "KOR", "KUW", "LAO", "LBN", "MDV", "MGL", "MYA", "NEP", "NGR", "OMA", "PAK", "PHI", "PRK", "QAT", "ROC", "SAU", "SIN", "SOL", "SRI", "SYR", "TJK", "THA", "TUR", "TKM", "TUV", "UAE", "UNK", "UZB", "VIE", "YAR", "YEM"
    ],
    "EU": [
        "ALB", "AND", "ARM", "AUT", "AZE", "BEL", "BIH", "BLR", "BUL", "CRO", "CYP", "CZE", "DEN", "EUN", "EST", "FIN", "FRA", "GER", "GEO", "GRE", "HUN", "ISL", "IRL", "ITA", "KOS", "LAT", "LIE", "LTU", "LUX", "MLT", "MNE", "MON", "NED", "NGR", "NOR", "POL", "POR", "ROU", "RUS", "SCG", "SRB", "SLO", "SMR", "SPN", "SWE", "SWZ", "SVK", "UKR", "URS"
    ],
    "NA": [
        "ANT", "ARU", "BAH", "BAR", "BER", "BIZ", "CAN", "CAY", "COK", "CRC", "CUB", "DMA", "DOM", "GRN", "GUA", "HAI", "HON", "IVB", "JAM", "LCA", "MTN", "NCA", "NFL", "PAN", "PLE", "PUR", "SKN", "TTO", "USA", "VIN"
    ],
    "OC": [
        "AUS", "FIJ", "FSM", "KIR", "MHL", "NRU", "NZL", "PLW", "PNG", "SAM", "SAA", "SOL", "TGA", "TUV", "VAN"
    ],
    "SA": [
        "ARG", "BOL", "BRA", "CHI", "COL", "ECU", "GUY", "PAR", "PER", "SUR", "URU", "VEN"
    ]
}

In [1]:
year_to_country_code = {
    1896: "GRE",  # Athens, Greece
    1900: "FRA",  # Paris, France
    1904: "USA",  # St. Louis, USA
    1908: "GBR",  # London, United Kingdom
    1912: "SWE",  # Stockholm, Sweden
    1916: None,   # Cancelled
    1920: "BEL",  # Antwerp, Belgium
    1924: "FRA",  # Paris, France
    1928: "NED",  # Amsterdam, Netherlands
    1932: "USA",  # Los Angeles, USA
    1936: "GER",  # Berlin, Germany
    1940: None,   # Cancelled
    1944: None,   # Cancelled
    1948: "GBR",  # London, United Kingdom
    1952: "FIN",  # Helsinki, Finland
    1956: "AUS",  # Melbourne, Australia
    1960: "ITA",  # Rome, Italy
    1964: "JPN",  # Tokyo, Japan
    1968: "MEX",  # Mexico City, Mexico
    1972: "GER",  # Munich, Germany
    1976: "CAN",  # Montreal, Canada
    1980: "URS",  # Moscow, Soviet Union
    1984: "USA",  # Los Angeles, USA
    1988: "KOR",  # Seoul, South Korea
    1992: "ESP",  # Barcelona, Spain
    1996: "USA",  # Atlanta, USA
    2000: "AUS",  # Sydney, Australia
    2004: "GRE",  # Athens, Greece
    2008: "CHN",  # Beijing, China
    2012: "GBR",  # London, United Kingdom
    2016: "BRA",  # Rio de Janeiro, Brazil
    2020: "JPN",  # Tokyo, Japan
    2024: "FRA",  # Paris, France
    2028: "USA",  # Los Angeles, USA
    2032: "AUS",  # Brisbane, Australia
}

In [7]:
hosts['NOC'] = hosts['Year'].map(year_to_country_code)

In [8]:
combdf = athletes.copy()

def ex_host_count(row):
  year = row['Year']
  noc = row['NOC']
  host_counts = 0
  for y in range(1896, year + 1, 4):  # Iterate through Olympic years
    if year_to_country_code.get(y) == noc:
      host_counts += 1
  return host_counts

def get_continent(noc):
  for continent, countries in continent_mapping.items():
    if noc in countries:
      return continent
  return None

def is_host_continent(row):
  year = row['Year']
  athlete_noc = row['NOC']
  host_noc = year_to_country_code.get(year)

  if host_noc is None:
    return 0

  athlete_continent = get_continent(athlete_noc)
  host_continent = get_continent(host_noc)

  if athlete_continent == host_continent:
    return 1
  else:
    return 0

def is_host_country(row):
  year = row['Year']
  noc = row['NOC']
  host_noc = year_to_country_code.get(year)

  if host_noc is None:
    return 0

  if noc == host_noc:
    return 1
  else:
    return 0

combdf['Is_Host'] = combdf.apply(is_host_country, axis=1)
combdf['Ex-Host'] = combdf.apply(ex_host_count, axis=1)
combdf['Host Continent'] = combdf.apply(is_host_continent, axis=1)

NameError: name 'continent_mapping' is not defined